<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/Bi-LSTM-GRU-intra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Dense, Bidirectional, GRU
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/intravehicle.csv')

In [ ]:
y_data = data.pop('4')
x_data = data

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x_data, y_data, test_size=0.2, random_state=0, shuffle=True)

In [ ]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [ ]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [ ]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(100, activation='tanh', recurrent_activation='sigmoid'), input_shape=(len(x_data.columns), 1)))
model.add(Dense(100, activation='ReLU'))
model.add(GRU(100, activation='tanh', recurrent_activation='sigmoid'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 200)              81600     
 l)                                                              
                                                                 
 dense (Dense)               (None, 4)                 804       
                                                                 
Total params: 82,404
Trainable params: 82,404
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_x, train_y, batch_size=32) 

414237/414237 [==============================] - 2051s 5ms/step - loss: 0.0317 - accuracy: 0.9893


In [ ]:
pred = model.predict(test_x, verbose=1)
pred_y = np.argmax(pred, axis=1)

103560/103560 [==============================] - 227s 2ms/step


In [ ]:
confusion_matrix(test_y, pred_y)

array([[ 117725,       0,       2,       0],
       [      0,   95547,    2652,       3],
       [  22750,    3431, 2821729,       0],
       [      0,       0,       0,  250056]])

In [ ]:
report = classification_report(test_y, pred_y, digits=5)
print(report)

              precision    recall  f1-score   support

           0    0.83805   0.99998   0.91188    117727
           1    0.96534   0.97296   0.96913     98202
           2    0.99906   0.99081   0.99492   2847910
           3    0.99999   1.00000   0.99999    250056

    accuracy                        0.99130   3313895
   macro avg    0.95061   0.99094   0.96898   3313895
weighted avg    0.99241   0.99130   0.99159   3313895

